In [ ]:

!pip install -q opencv-python opencv-python-headless torch torchvision matplotlib tqdm

!mkdir -p model

!rm -f model/srcnn_x4.pth
!wget -q -O model/srcnn_x4.pth "https://www.dropbox.com/s/pd5b2ketm0oamhj/srcnn_x4.pth?dl=1"

from google.colab import files
import os

print("Upload your video 🎥")
uploaded = files.upload()
video_path = list(uploaded.keys())[0]



In [ ]:
os.makedirs('frames', exist_ok=True)
!ffmpeg -hide_banner -loglevel error -i "$video_path" -qscale:v 1 frames/frame%04d.png

import torch.nn as nn
import torch

class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()
        # The model architecture remains the same
        self.conv1 = nn.Conv2d(1, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(32, 1, kernel_size=5, padding=2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x

from glob import glob
from tqdm import tqdm
import cv2
import numpy as np
from torchvision.transforms import ToTensor

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = SRCNN().to(device)
model.load_state_dict(torch.load('model/srcnn_x4.pth', map_location=device))
model.eval()

input_dir = 'frames'
output_dir = 'frames_upscaled'
os.makedirs(output_dir, exist_ok=True)

frame_list = sorted(glob(os.path.join(input_dir, '*.png')))
to_tensor = ToTensor()

for frame_path in tqdm(frame_list, desc="Upscaling Frames (4x)"):
    img = cv2.imread(frame_path)
    img_ycrcb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(img_ycrcb)

    y_resized = cv2.resize(y, (y.shape[1]*4, y.shape[0]*4), interpolation=cv2.INTER_CUBIC)

    y_tensor = to_tensor(y_resized).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(y_tensor).clamp(0, 1)

    out_y = out.squeeze().cpu().numpy() * 255.0
    out_y = out_y.astype(np.uint8)

    cr = cv2.resize(cr, (out_y.shape[1], out_y.shape[0]), interpolation=cv2.INTER_CUBIC)
    cb = cv2.resize(cb, (out_y.shape[1], out_y.shape[0]), interpolation=cv2.INTER_CUBIC)

    out_img = cv2.merge([out_y, cr, cb])
    out_bgr = cv2.cvtColor(out_img, cv2.COLOR_YCrCb2BGR)

    output_frame_path = os.path.join(output_dir, os.path.basename(frame_path))
    cv2.imwrite(output_frame_path, out_bgr)

output_video = "upscaled_output_srcnn_x4.mp4"
framerate = !ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1:nokey=1 "$video_path"
framerate = framerate[0] if framerate else '30' # Default to 30 if not found

print(f"Reconstructing video with framerate: {framerate}")
!ffmpeg -hide_banner -loglevel error -y -framerate {framerate} -i frames_upscaled/frame%04d.png -c:v libx264 -crf 18 -preset slow -pix_fmt yuv420p "$output_video"

print(f"\nDone! Downloading {output_video}")
from google.colab import files
files.download(output_video)